In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.data.path.append('/home/sujay1844/.local/share/nltk_data/')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
# from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, mean_absolute_error

2023-04-01 11:38:31.503959: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
# Load the dataset
df = pd.read_csv(
	'./train_snli.txt.zip',
	sep='\t',
	header=None,
	names=['sentence1', 'sentence2', 'label']
)[:10_000]
df.head()

,sentence1,sentence2,label
0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",0
1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
2,Children smiling and waving at camera,There are children present,1
3,Children smiling and waving at camera,The kids are frowning,0
4,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0


In [ ]:
# Load the dataset
df = pd.read_csv(
	'./train_snli.txt.zip',
	sep='\t',
	header=None,
	names=['sentence1', 'sentence2', 'label']
)[:10_000]
df.head()

,sentence1,sentence2,label
0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",0
1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
2,Children smiling and waving at camera,There are children present,1
3,Children smiling and waving at camera,The kids are frowning,0
4,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0


In [2]:
stop_words = set(stopwords.words('english'))
# stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):

    # Remove punctuations and digits
    try:
        text = re.sub('[^a-zA-Z\s]', '', text)
    except Exception as e:
        print("Regex failed", text)
        print(e)
        text = 'foo'

    text = text.lower()

    # Some sentences are empty after removing punctuations and digits, which causes errors
    try:
        words = nltk.word_tokenize(text)
    except Exception as e:
        print("Tokenization failed", text)
        print(e)
        words = []

    # Remove stop words and stem
    # words = [stemmer.stem(word) for word in words if word not in stop_words]

    # Remove stop words and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Join the words back into a sentence
    text = ' '.join(words)
    return text


In [ ]:

# Apply the preprocess_text function to each sentence pair in the dataset
df['sentence1'] = df['sentence1'].apply(preprocess_text)
df['sentence2'] = df['sentence2'].apply(preprocess_text)

In [24]:
# Vectorize the text data using Bag-of-Words (BoW)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['sentence1'] + df['sentence2']).toarray()
y = tf.keras.utils.to_categorical(df['label'])

In [25]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

2023-03-24 14:31:48.571174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 14:31:48.647217: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [27]:
# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=10
)

Epoch 1/10
250/250 [==============================] - 8s 29ms/step - loss: 0.6926 - accuracy: 0.5080 - val_loss: 0.6900 - val_accuracy: 0.5005
Epoch 2/10
250/250 [==============================] - 6s 25ms/step - loss: 0.6253 - accuracy: 0.6553 - val_loss: 0.7451 - val_accuracy: 0.4835
Epoch 3/10
250/250 [==============================] - 5s 21ms/step - loss: 0.4670 - accuracy: 0.7779 - val_loss: 1.0330 - val_accuracy: 0.4675
Epoch 4/10
250/250 [==============================] - 6s 22ms/step - loss: 0.2899 - accuracy: 0.8771 - val_loss: 1.4047 - val_accuracy: 0.4630
Epoch 5/10
250/250 [==============================] - 6s 24ms/step - loss: 0.1668 - accuracy: 0.9284 - val_loss: 1.8484 - val_accuracy: 0.4750
Epoch 6/10
250/250 [==============================] - 7s 27ms/step - loss: 0.1042 - accuracy: 0.9563 - val_loss: 2.1388 - val_accuracy: 0.4625
Epoch 7/10
250/250 [==============================] - 7s 27ms/step - loss: 0.0735 - accuracy: 0.9712 - val_loss: 2.3240 - val_accuracy: 0.4710

In [3]:
# Save the model
import pickle
# pickle.dump(model, open('model.pkl', 'wb'))

In [5]:
# Load the model
model = pickle.load(open('./plagiarism-checker-50k.pkl', 'rb'))

ReadError: file could not be opened successfully:
- method gz: ReadError('not a gzip file')
- method bz2: ReadError('not a bzip2 file')
- method xz: ReadError('not an lzma file')
- method tar: ReadError('invalid header')

In [ ]:
# test with own data
para1 = "Normal science, the activity in which most scientists inevitably spend almost all their time, is predicated on the assumption that the scientific community knows what the world is like. Much of the success of the enterprise derives from the community's willingness to defend that assumption, if necessary at considerable cost. Normal science, for example, often suppresses fundamental novelties because they are necessarily subversive of its basic commitments"
para2 = "Normal science, the activity in which most scientists inevitably spend almost all their time, is predicated on the assumption that the scientific community knows what the world is like. Some scientists say that the success of the enterprise comes from the community’s willingness to defend that assumption, if necessary at considerable cost. Normal science often suppresses fundamental novelties because they are necessarily subversive of its basic commitments."

para1 = nltk.sent_tokenize(para1)
para2 = nltk.sent_tokenize(para2)
preds = []
for sent1 in para1:
	single_pred = []
	sent1 = preprocess_text(sent1)
	for sent2 in para2:
		sent2 = preprocess_text(sent2)
		X_test = vectorizer.transform([sent1 + sent2]).toarray()
		y_pred = model.predict(X_test)
		print((sent1, sent2))
		print(y_pred[0])
		single_pred.append(y_pred[0])
	# print(single_pred)
	preds.append(single_pred)

1/1 [==============================] - 0s 30ms/step
('normal science activity scientist inevitably spend almost time predicated assumption scientific community know world like', 'normal science activity scientist inevitably spend almost time predicated assumption scientific community know world like')
[2.1033704e-09 1.0000000e+00]
1/1 [==============================] - 0s 55ms/step
('normal science activity scientist inevitably spend almost time predicated assumption scientific community know world like', 'scientist say success enterprise come community willingness defend assumption necessary considerable cost')
[2.9003066e-05 9.9997103e-01]
1/1 [==============================] - 0s 33ms/step
('normal science activity scientist inevitably spend almost time predicated assumption scientific community know world like', 'normal science often suppresses fundamental novelty necessarily subversive basic commitment')
[2.9595711e-04 9.9970406e-01]
1/1 [==============================] - 0s 49ms/

In [ ]:
print(preds)

[[array([1.2329283e-06, 9.9999881e-01], dtype=float32), array([0.03193902, 0.96806103], dtype=float32), array([0.20104283, 0.7989572 ], dtype=float32)], [array([0.00524586, 0.9947542 ], dtype=float32), array([0.5131527 , 0.48684728], dtype=float32), array([0.9760982 , 0.02390184], dtype=float32)], [array([0.06701543, 0.9329846 ], dtype=float32), array([0.95893884, 0.04106119], dtype=float32), array([0.99461776, 0.0053823 ], dtype=float32)]]


The following numbers are average of three runs with a dataset of 10,000 pairs of sentences.

| Model | Accuracy |
|--|--|
| Base | 0.9872 |
| Stemmed | 0.9866 |
| Lemmetized | 0.9882 |